In [1]:
import polars as pl
import os
import random

In [32]:
hospital_data = "D:/GitHub/THCICAnalysis/Hospital/"
random_df = os.path.join(hospital_data, random.choice(os.listdir(hospital_data))).replace('\\','/')
random_df

'D:/GitHub/THCICAnalysis/Hospital/PUDF_base1_4q2005_tab.csv'

In [3]:
hospital_df = pl.read_csv(random_df, infer_schema_length=0)

In [34]:
zipPop = "../assets/census/interpolatedPopulations.csv"

full_zip_population = pl.read_csv(zipPop)
full_zip_population

PAT_ZIP,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
73949,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1866,1764,1662,1561,1459,1357
75001,8175,8569,8963,9357,9751,10145,10540,10934,11328,11722,12116,12510,12892,13274,13657,14039,14421,14794,15167,15541,15914,16287
75002,37171,39444,41717,43990,46263,48536,50810,53083,55356,57629,59902,62175,63513,64850,66188,67525,68863,69311,69759,70206,70654,71102
75006,45990,45985,45979,45974,45969,45963,45958,45952,45947,45942,45936,45931,46752,47574,48395,49217,50038,49651,49264,48878,48491,48104
75007,50357,50408,50460,50511,50562,50613,50665,50716,50767,50818,50870,50921,51599,52277,52956,53634,54312,54339,54366,54394,54421,54448
75009,4913,5232,5550,5869,6188,6506,6825,7143,7462,7781,8099,8418,8778,9139,9499,9860,10220,13123,16027,18930,21834,24737
75010,13141,14015,14889,15763,16637,17511,18386,19260,20134,21008,21882,22756,23352,23949,24545,25142,25738,26616,27494,28373,29251,30129
75013,11584,13082,14579,16077,17575,19073,20570,22068,23566,25064,26561,28059,29957,31855,33752,35650,37548,39000,40452,41903,43355,44807
75019,35805,36077,36349,36621,36893,37165,37437,37709,37981,38253,38525,38797,39210,39623,40037,40450,40863,41555,42247,42940,43632,44324


In [19]:
def interpCensus(quarter):
    year = quarter[2:]
    df = zip_population.select(
        ['PAT_ZIP',year]
    )
    df = df.with_column({
        'PAT_ZIP': 'PAT_ZIP',
        year: 'population'
    })

    return df

In [21]:
import pandas as pd

icd = pd.read_pickle('../assets/ICDPickle.pkl')
icd = pl.DataFrame._from_pandas(icd)
icd

PRINC_DIAG_CODE,Count,ICD-9,icd9cm_mult
str,f64,str,list[str]
"""Z3800""",3.940227e6,"""V3900""","[""V3000"", ""V3900""]"
"""Z3801""",2.099687e6,"""V3001""","[""V3001""]"
"""A419""",1.211138e6,"""99591""","[""0389"", ""99591""]"
"""J189""",820711.0,"""486""","[""486""]"
"""Z5189""",676973.0,"""V589""","[""V5889"", ""V589""]"
"""O3421""",627506.0,"""65423""","[""65421"", ""65423""]"
"""J441""",500761.0,"""49322""","[""49121"", ""49322""]"
"""I2510""",488267.0,"""4292""","[""41401"", ""4292""]"
"""N179""",470121.0,"""5849""","[""5849""]"


In [132]:
def loopICDPerQuarter(time_period, nthresh=1, icd10=True):
    data = pl.DataFrame()
    for ind, quarter in enumerate(time_period):
        print(quarter)
        hospital_df = pl.read_csv(f"{hospital_data}PUDF_base1_{quarter}_tab.csv", infer_schema_length=0)
        hospital_df = (hospital_df
                        # .filter(~(pl.col('PAT_ZIP').str.ends_with('.0')))
                        .with_columns(pl.col('PAT_ZIP').str.replace(r'\.0$',''))
                        .filter(pl.col('PAT_ZIP') != '`')
                        .filter(pl.col('PAT_ZIP').str.lengths() == 5)
                        .with_columns(pl.col('PAT_ZIP').cast(pl.Int64))
        )
        # print(hospital_df.glimpse())
        # zip_population = interpCensus(quarter)
        year = quarter[2:]
        zip_population = (full_zip_population
                          .select(['PAT_ZIP',year])
                          .rename({year: 'population'})
        )
        # print(zip_population.glimpse())

        if icd10 == True:
            codes = icd['PRINC_DIAG_CODE']
        else:
            codes = icd['ICD-9']

        for sind, icd_code in enumerate(codes[:2]):
            os.makedirs(f'icd10/{icd["PRINC_DIAG_CODE"][sind]}', exist_ok=True)

            base_df = pl.DataFrame()
            if icd10 == False:
                for item in icd['icd9cm_mult'][sind]: 
                    print(icd_code)
                    icd_df = (hospital_df
                              .filter(hospital_df['PRINC_DIAG_CODE'] == item)
                              .group_by(['PAT_ZIP']).count()
                              .sort('count', descending=True)
                              .rename({'count': 'ICD'})
                              .join(zip_population, on='PAT_ZIP')
                              .with_columns((pl.col('ICD')/pl.col('population'))
                                            .alias('normalized'))                        
                              .filter(pl.col('ICD') >= nthresh)
                    )
                    print(icd_df.glimpse())

                    base_df = pl.concat([base_df, icd_df])
            else:
                icd_df = (hospital_df
                              .filter(hospital_df['PRINC_DIAG_CODE'] == icd_code)
                              .group_by(['PAT_ZIP']).count()
                              .sort('count', descending=True)
                              .rename({'count': 'ICD'})
                              .join(zip_population, on='PAT_ZIP')
                              .with_columns((pl.col('ICD')/pl.col('population'))
                                            .alias('normalized'))                              
                              .filter(pl.col('ICD') >= nthresh)
                    )
            base_df.write_csv(f'./icd10/{icd["PRINC_DIAG_CODE"][sind]}/{quarter}.csv')
        
            # del hospital_df
    del hospital_df

    return data



In [57]:
start_year = 2005
end_year = 2022

hospital_quarters = [f"{quarter}q{year}" for year in range(start_year, end_year + 1) for quarter in range(1, 5) if not (year == end_year and quarter > 2)]
hospital_quarters = hospital_quarters[:-1]
icd9_subset = hospital_quarters[hospital_quarters.index(f'1q{start_year}'):hospital_quarters.index('4q2015')]
icd10_subset = hospital_quarters[hospital_quarters.index('4q2015'):hospital_quarters.index(f'1q{end_year}')]

In [133]:
loopICDPerQuarter(icd9_subset[:1], icd10=False)

1q2005
V3900
Rows: 1421
Columns: 4
$ PAT_ZIP    <i64> 75001, 75002, 75006, 75007, 75009, 75010, 75013, 75019, 75020, 75021
$ ICD        <u32> 44, 130, 143, 92, 9, 47, 64, 75, 44, 8
$ population <i64> 10145, 48536, 45963, 50613, 6506, 17511, 19073, 37165, 22167, 7920
$ normalized <f64> 0.004337111877772301, 0.0026784242624031647, 0.0031111981376324436, 0.0018177148163515302, 0.0013833384568090992, 0.0026840271829135973, 0.003355528757930058, 0.002018027714247276, 0.001984932557405152, 0.00101010101010101

None
V3900
Rows: 0
Columns: 4
$ PAT_ZIP    <i64> 
$ ICD        <u32> 
$ population <i64> 
$ normalized <f64> 

None
V3001
Rows: 1317
Columns: 4
$ PAT_ZIP    <i64> 75001, 75002, 75006, 75007, 75009, 75010, 75013, 75019, 75020, 75021
$ ICD        <u32> 27, 96, 44, 53, 4, 24, 39, 41, 21, 3
$ population <i64> 10145, 48536, 45963, 50613, 6506, 17511, 19073, 37165, 22167, 7920
$ normalized <f64> 0.002661409561360276, 0.0019779133014669525, 0.0009572917346561365, 0.001047161796376425, 0.00061

shape: (0, 0)
┌┐
╞╡
└┘

In [16]:
temp_df = (hospital_df
 .filter(~(pl.col('PAT_ZIP').str.ends_with('.0')))
 .filter(pl.col('PAT_ZIP') != '`')
 .filter(pl.col('PAT_ZIP').str.lengths() == 5)
 .with_columns(pl.col('PAT_ZIP').cast(pl.Int64))
 .filter(pl.col('SEX_CODE') == 'M')
 .filter(pl.col('PRINC_DIAG_CODE') == 'A4189')
.group_by(["PAT_ZIP"]).count()
.sort('count', descending=True)
 )
temp_df

PAT_ZIP,count
i64,u32
79601,12
79603,8
77469,5
79604,4
88888,4
78660,4
79510,4
78669,4
76712,3


In [11]:
def groupDF(df, icd_code):
    '''
    This function takes in a Polars DataFrame, finds out which patients have the given icd code,
    then counts by zip code and returns this groupby DataFrame.
    '''
    # Convert the column to String type if not already
    df = df.with_column(df['PRINC_DIAG_CODE'].cast(pl.Object))
    
    # Filter rows with the specified icd_code
    filtered_df = df.filter(df['PRINC_DIAG_CODE'] == icd_code)
    
    # Group by 'PAT_ZIP' and count the occurrences, then sort by count in descending order
    grouped_df = filtered_df.groupby('PAT_ZIP').agg(pl.sum('PRINC_DIAG_CODE').alias('ICD')).sort('ICD', reverse=True)
    
    return grouped_df

# Example usage:
# Assuming you have a Polars DataFrame named 'df'

In [14]:
type(temp_df)

polars.dataframe.frame.DataFrame

In [13]:
groupDF(temp_df,'A4189')

AttributeError: 'DataFrame' object has no attribute 'with_column'

In [6]:
# lf = pl.LazyFrame({"a": [1, 2, 3], "b": ["x", None, "z"]})
lf = pl.scan_csv(random_df, infer_schema_length=0)

res = pl.SQLContext(frame=lf).execute(

    "SELECT PAT_ZIP, SEX_CODE, COUNT(*) as count FROM frame WHERE SEX_CODE = 'M' GROUP BY PAT_ZIP;"

)

res.collect()

PAT_ZIP,SEX_CODE,count
str,list[str],u32
"""76201""","[""M"", ""M"", … ""M""]",168
"""75933.0""","[""M"", ""M"", … ""M""]",8
"""79022.0""","[""M"", ""M"", … ""M""]",52
"""78121""","[""M"", ""M"", … ""M""]",101
"""75931.0""","[""M"", ""M"", … ""M""]",17
"""76050.0""","[""M"", ""M"", … ""M""]",10
"""77426.0""","[""M"", ""M"", … ""M""]",8
"""77447.0""","[""M"", ""M"", … ""M""]",48
"""76679""","[""M"", ""M"", … ""M""]",9
